In [1]:
!pip install transformers datasets torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Wed Jul 26 08:07:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import torchvision.transforms as transforms #이미지 크기 변환에 사용, 혹은 데이터 증강에 사
from transformers import ViTModel, ViTConfig, ViTForImageClassification
from datasets import load_dataset
from torch.utils.data import DataLoader

In [4]:
# Load the CIFAR-10 dataset
dataset = load_dataset("cifar10")

# Define the subset size
subset_size = 1000  #1000개만 사용 이미

# Take a subset of the dataset
dataset = dataset["train"].select(range(subset_size))

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/features/image.py:327: UserWarning: Downcasting array dtype uint8 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['img', 'label'],
    num_rows: 1000
})

In [6]:
# Define the ViT model
config = ViTConfig.from_pretrained("google/vit-large-patch32-384")
# 해당 구성을 사용하여 model 로
model = ViTForImageClassification(config)

In [7]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(32, 32), stride=(32, 32))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_f

In [8]:
# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [9]:
# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(32, 32), stride=(32, 32))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_f

In [10]:
device

device(type='cuda')

In [11]:
# Define the batch size and number of training epochs
batch_size = 8
num_epochs = 10

## image를 384*384로 사용하기 위해 사용

In [12]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((384, 384)),  # Resize the images
    transforms.ToTensor(),
])

In [13]:
# Custom collate function to handle PIL images
def custom_collate(batch):
  ## batch전달받고 image와 label을 위 transform을 사용하여 변환
    images = [transform(item["image"]) for item in batch]
    labels = [item["label"] for item in batch]
    return {"image": torch.stack(images), "label": torch.tensor(labels)}

In [14]:
dataset

Dataset({
    features: ['img', 'label'],
    num_rows: 1000
})

In [15]:
# Preprocess the dataset
## 이걸 해줘야하는건가? image와 label 매핑
preprocessed_dataset = dataset.map(lambda x: {'image': x['img'], 'label': x['label']})

# Create a data loader for the preprocessed dataset with custom collate function
dataloader = DataLoader(preprocessed_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
len(dataloader)

125

In [17]:
1000 // 8

125

In [ ]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        # Move batch to device
        inputs = batch["image"].to(device)
        labels = batch["label"].to(device)

        # Forward pass
        outputs = model(inputs)
        logits = outputs.logits

        # Calculate loss
        loss = loss_fn(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

Epoch 1/10, Average Loss: 2.7641
Epoch 2/10, Average Loss: 2.3003
Epoch 3/10, Average Loss: 2.2598
Epoch 4/10, Average Loss: 2.1051
Epoch 5/10, Average Loss: 2.0263
Epoch 6/10, Average Loss: 1.9901
Epoch 7/10, Average Loss: 1.9801
Epoch 8/10, Average Loss: 1.8544
Epoch 9/10, Average Loss: 1.7962
